In [0]:
#define parameters
INP_SRC_SYS="bgmax"
SRC_STREAM_ID="BGMAX001"
PARTITION_DATE="FROM_FILE"
WRITE_MODE="Overwrite"
ENV="dev"
AZURE_URL="dbfs://"+ENV
import datetime
date_time=datetime.datetime.now().strftime("%Y%m%d%H%M%S")
CREATE_DATE= datetime.datetime.now().strftime("%Y-%m-%d")
print(CREATE_DATE)
current_year=CREATE_DATE.split("-")[0].lstrip('0')
current_month=CREATE_DATE.split("-")[1].lstrip('0')
current_day=CREATE_DATE.split("-")[2].lstrip('0')
BATCH_ID = SRC_STREAM_ID+"_"+date_time
print(BATCH_ID)
print(date_time)

In [0]:
import os
import re
def db_list_files(file_path, file_prefix):
  #file_list = [file.path for file in dbutils.fs.ls(file_path) if os.path.basename(file.path).startswith(file_prefix)]
  file_list = [file.path for file in dbutils.fs.ls(file_path) if re.search(file_prefix, os.path.basename(file.path))]
  return file_list
#db_list_files(AZURE_URL+"/tmp/data/payment/bgmax/", "^bgmax_bettdfi_[0-9]{8}_[0-9]")

In [0]:
def meta_model(INP_SRC_SYS):
  try:
    metamodel_full_df = (spark.read           # The DataFrameReader
       .option("sep", "|")        # Use tab delimiter (default is comma-separator)
      #.option("header", "true")   # Use first line of all files as header
       .schema(METAMODEL_SCHEMA)   #Apply schema
       .csv(METAMODEL_LOOKUP)               # Creates a DataFrame from CSV after reading in the file
    )
    #metamodel_full_df.show(vertical=True)
    metamodel_df=metamodel_full_df.filter(metamodel_full_df.SRC_STREAM_ID==SRC_STREAM_ID)
    metamodel_df.show(vertical=True)
    return metamodel_df
  except:
    print("Metamodel doesnt exists for this source")
#metamodel_df.show()

In [0]:
def file_exists(path):
  try:
    dbutils.fs.ls(path)
    print("The path ",path," exists")
    pass
    return True
  except:
    print("The path ",path," does not exist")
    return False

In [0]:
#define Dropzone to Raw pipeline
def dropzone_to_raw(INP_DF):
  print("###########################DropZone to RAW Pipeline ##################################")
  #INP_DF.show(vertical=True)
  #input_file=INP_DF.select("STREAM_NM").rdd.flatMap(list).collect()
  input_file=INP_DF.select("STREAM_NM").rdd.flatMap(list).collect()[0]
  LANDING_DIR=INP_DF.select("LANDING_DIR").rdd.flatMap(list).collect()[0]
  Azure_land_dir=LANDING_DIR.replace("/mnt/shareddisk",AZURE_URL+"/tmp")
  RAW_ODL_LOC=INP_DF.select("RAW_ODL_LOC").rdd.flatMap(list).collect()[0]
  Azure_raw_dir=AZURE_URL+"/raw" + RAW_ODL_LOC + CREATE_DATE.replace("-",'')
  
  print("input file name is :",input_file)
  print("Landing directory is :",LANDING_DIR)
  print("Raw directory is :",RAW_ODL_LOC)
  print("Azure Raw Loc is :",Azure_raw_dir)
  #File listing
#  print(dbutils.fs.ls(Azure_land_dir))
  landing_file = db_list_files(Azure_land_dir, input_file)[0]
  raw_file_name = landing_file.split(os.sep)[-1]
  #check if directory exists, else create it
  if not file_exists(Azure_raw_dir):
    print("Creating directory ",Azure_raw_dir,"now")
    dbutils.fs.mkdirs(Azure_raw_dir)
  Azure_raw_file = Azure_raw_dir +"/"+ raw_file_name
  print("RAW FILE:",Azure_raw_file)
  if not file_exists(Azure_raw_file):
  # Copy the file from Landing directory to Raw folder  
    print("Copying the file ",landing_file,"to Raw ADLS ",Azure_raw_dir)
    dbutils.fs.cp(landing_file,Azure_raw_dir)
    
  return Azure_raw_file

In [0]:
def create_partions_with_current_date(base_dir,part_keys):
  #part_keys="ing_year=YYYY:ing_month=MM:ing_day=DD"
  partition_values = {}
  partitons=part_keys.split(":")
  #len=len(xx)
  partition_k = base_dir
  for i in partitons:
    #print(i)
    if i == 'ing_year=YYYY' :
      partition_k = partition_k + '/ing_year='+current_year
      partition_values['ing_year']=current_year
      if not file_exists(partition_k):
        print("Creating directory ",partition_k,"now")
        #dbutils.fs.mkdirs(partition_k)
    elif i == 'ing_month=MM':
      partition_k = partition_k + '/ing_month='+current_month
      partition_values['ing_month']=current_month
      if not file_exists(partition_k):
        print("Creating directory ",partition_k,"now")
        #dbutils.fs.mkdirs(partition_k)
    elif i == 'ing_day=DD':
      partition_k = partition_k + '/ing_day='+current_day
      partition_values['ing_day']=current_day
      if not file_exists(partition_k):
        print("Creating directory ",partition_k,"now")
        #dbutils.fs.mkdirs(partition_k)
  print(partition_k)
  print(partition_values)
  return partition_k

In [0]:
def create_partions_from_file(base_dir,part_keys):
  #part_keys="ing_year=YYYY:ing_month=MM:ing_day=DD"
  partition_values = {}
  partitons=part_keys.split(":")
  #len=len(xx)
  partition_k = base_dir
  for i in partitons:
    #print(i)
    if i == 'ing_year=YYYY' :
      partition_k = partition_k + '/ing_year='+current_year
      partition_values['ing_year']=current_year
      if not file_exists(partition_k):
        print("Creating directory ",partition_k,"now")
        #dbutils.fs.mkdirs(partition_k)
    elif i == 'ing_month=MM':
      partition_k = partition_k + '/ing_month='+current_month
      partition_values['ing_month']=current_month
      if not file_exists(partition_k):
        print("Creating directory ",partition_k,"now")
        #dbutils.fs.mkdirs(partition_k)
    elif i == 'ing_day=DD':
      partition_k = partition_k + '/ing_day='+current_day
      partition_values['ing_day']=current_day
      if not file_exists(partition_k):
        print("Creating directory ",partition_k,"now")
        #dbutils.fs.mkdirs(partition_k)
  print(partition_k)
  print(partition_values)
  return partition_k

In [0]:
#define Raw to Base pipeline
from pyspark.sql.functions import lit
def raw_to_base(INP_DF,INP_FILE):
  print("###########################RAW to BASE Pipeline ##################################")
  #INP_DF.show(vertical=True)
  #input_file=INP_DF.select("STREAM_NM").rdd.flatMap(list).collect()
  BASE_ODL_LOC=INP_DF.select("BASE_ODL_LOC").rdd.flatMap(list).collect()[0]
  BASE_TBL_NAME = INP_DF.select("BASE_TBL_NAME").rdd.flatMap(list).collect()[0]
  PARTITION_KEY = INP_DF.select("PARTITION_KEY").rdd.flatMap(list).collect()[0]
  FIELD_SEP = INP_DF.select("FIELD_SEP").rdd.flatMap(list).collect()[0]
  Azure_base_dir=AZURE_URL+"/base" + BASE_ODL_LOC + BASE_TBL_NAME.split("_")[1]
  
  #Create partition directories
  if PARTITION_DATE=='FROM_FILE':
    base_dir=create_partions_from_file(Azure_base_dir,PARTITION_KEY)
  else:
    base_dir=create_partions_with_current_date(Azure_base_dir,PARTITION_KEY)
    
  print("BASE_TBL_NAME  is :",BASE_TBL_NAME)
  print("BASE_ODL_LOC  is :",Azure_base_dir)
  print("PARTITION_KEY  is :",PARTITION_KEY)
  print("Base Dir is :",base_dir)
  print("INP_FILE is :",INP_FILE)
  
  #Read Input file
  raw_df = (spark.read
            .option("sep",FIELD_SEP)
            .option("header",True)
            .csv(INP_FILE)
            )
  #raw_df.show()
  repartitionedDF = raw_df.repartition(4)
  enrich_df=repartitionedDF.select(lit(SRC_STREAM_ID).alias("src_stream_id"),
                          lit(BATCH_ID).alias("batch_id"),
                          lit(CREATE_DATE).alias("create_date"),
                          lit("UTC").alias("time_zone"),
                          lit(CREATE_DATE).alias("src_business_date"),
                          "*")
  #enrich_df.show()
  print("Writing file ", base_dir, "with ",enrich_df.rdd.getNumPartitions()," partitions")
  for k in partition_values:
    #print(k,d[k])
    #print(k)
      enrich_df=enrich_df.withColumn(k,lit(partition_values[k]))
#enrich_df.printSchema()
  enrich_df.write.format("avro").partitionBy(list(partition_values.keys())).mode(WRITE_MODE).save(base_dir)



In [0]:
# Metamodel lookup reading
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
#import sys
#defining metamodel Schema
METAMODEL_SCHEMA=StructType([ \
    StructField("PROCESS_ID",StringType(),True), \
    StructField("PROCESS_TYPE",StringType(),True), \
    StructField("SRC_ID",StringType(),True), \
    StructField("SRC_SYS_NM",StringType(),True), \
    StructField("STREAM_NM",StringType(),True), \
    StructField("SRC_STREAM_ID",StringType(),True), \
    StructField("SRC_SYS_TYPE",StringType(),True), \
    StructField("SRC_SYS_DESC",StringType(),True), \
    StructField("FIELD_SEP",StringType(),True), \
    StructField("LINE_SEP",StringType(),True), \
    StructField("HEADER_ROW",StringType(),True), \
    StructField("FOOTER_ROW",StringType(),True), \
    StructField("LOAD_TYPE",StringType(),True), \
    StructField("WHERE_CLUASE",StringType(),True), \
    StructField("PARTITION_KEY",StringType(),True), \
    StructField("CUST_DATA_FLG",StringType(),True), \
    StructField("CUST_DATA_DESC",StringType(),True), \
    StructField("LEGAL_GRAND",StringType(),True), \
    StructField("LOAD_FREQ",StringType(),True), \
    StructField("LANDING_DIR",StringType(),True), \
    StructField("RAW_ODL_LOC",StringType(),True), \
    StructField("BASE_TBL_NAME",StringType(),True), \
    StructField("BASE_ODL_LOC",StringType(),True), \
    StructField("SCHEMA_DIR",StringType(),True), \
    StructField("C1_TBL_NAME",StringType(),True), \
    StructField("C1_ODL_LOC",StringType(),True), \
    StructField("DELIVERY_FORMAT",StringType(),True), \
    StructField("SRV_NAME",StringType(),True), \
    StructField("SRV_SHRT_NAME",StringType(),True), \
    StructField("SRC_TIME_ZONE",StringType(),True) \
  ])

try:
  METAMODEL_LOOKUP = AZURE_URL+"/lookup/"+INP_SRC_SYS+"_meta_config.lkp"
  #Read lookup files
  #print(dbutils.fs.ls(METAMODEL_LOOKUP))
  #Call Metamodel to fetch data from metamodel lookup
  metamodel_df=meta_model(INP_SRC_SYS)
  #metamodel_df.show(vertical=True)
  metamodel_src2dropzone_df=metamodel_df.filter(metamodel_df.PROCESS_TYPE=="SOURCE_TO_DROPZONE")
  metamodel_dropzone2raw_df=metamodel_df.filter(metamodel_df.PROCESS_TYPE=="DROPZONE_TO_RAW")
  metamodel_raw2base_df=metamodel_df.filter(metamodel_df.PROCESS_TYPE=="RAW_TO_BASE")
  #Dropzone to Raw
  Azure_raw_file = dropzone_to_raw(metamodel_dropzone2raw_df)
  #Raw to Base
  raw_to_base(metamodel_raw2base_df,Azure_raw_file)
  #metamodel_raw2base_df.show(vertical=True)

except:
  print("Metamodel lookup file",METAMODEL_LOOKUP," doesnt exists for this source")
        #ys.exc_info()[0], "occurred.")

